<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.33.0


    Uninstalling protobuf-6.33.0:
      Successfully uninstalled protobuf-6.33.0


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.66
    Uninstalling yfinance-0.2.66:


      Successfully uninstalled yfinance-0.2.66
   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-10-20 15:04:03
-------------------
qualified stocks: 86
with latest results: 27
still star stocks: 16
-------------------
Initial Investment:  1.23 C
CY Investment:  1.50 C
Reserve:  1.09 L
Current:  1.41 C
-------------------
Today PnL: 57.74 K (0.41%)
Current PnL: -20.57 L (-13.7%)
CY Booked + Current PnL: -8.67 L (-5.78%)
-------------------
Total profit:  2.10 L
Total loss:  -22.67 L
-------------------
Total Booked + Current PnL: 18.31 L (14.87%)
Total Booked PnL: 38.88 L (31.58%)
Curr Year Booked PnL: 11.90 L (8.45%)
Prev Year Booked PnL: 26.98 L (21.91%)
Est FTT:  2.29 C
Est FTT PnL: 87.74 L (62.26%)
Deployed:  1.23 C
Current:  1.41 C
CAGR/XIRR %: 7.98%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
18,CIPLA,1662.74,-16.01,71.0,X-LC,4.02,224406.0,19906.0,3389.0,3.83,9.73,1.51,11.39,10.0,5.87,1.60,19.56,X40N,ATH,PHARMA
56,RELIANCE,1533.00,-8.78,73.0,X-LC,2.89,228415.0,18049.0,10736.0,3.35,8.58,4.70,13.68,37.0,1.68,1.63,26.52,XY25,NTT,REFINERIES
83,VOLTAS,1530.00,3.12,63.0,X-MC,1.57,216360.0,24618.0,13133.0,1.11,12.84,6.07,19.69,99.0,1.87,1.55,20.51,XY25,NTT,AC
76,TTKPRESTIG,770.00,94.24,44.0,M-SC,1.73,83683.0,-17094.0,17188.0,-0.53,-16.96,20.54,0.09,245.0,-0.99,0.60,9.94,OX40N,NTT,DURABLES
33,ICICIGI,2260.25,-13.61,65.0,X-MC,2.65,148577.0,12444.0,18676.0,0.34,9.14,12.57,22.86,91.0,0.67,1.06,22.91,X40,ATH,INSURANCE


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
68,SURYODAY,240.00,66.67,61.0,H-SC,11.16,151980.0,-27091.0,92814.0,5.46,-15.13,61.07,36.71,135.0,-0.29,1.09,50.79,XR,NTT,BANKS
11,BANKINDIA,190.00,-22.98,65.0,H-MC,12.57,199248.0,19440.0,92969.0,5.23,10.81,46.66,62.52,88.0,0.21,1.43,48.58,XR,NTT,BANKS
77,UJJIVANSFB,60.00,118.20,69.0,H-SC,12.95,137916.0,-4563.0,24080.0,4.84,-3.20,17.46,13.70,163.0,-0.19,0.99,62.78,OX40N,NTT,BANKS
18,CIPLA,1662.74,-16.01,71.0,X-LC,4.02,224406.0,19906.0,3389.0,3.83,9.73,1.51,11.39,10.0,5.87,1.60,19.56,X40N,ATH,PHARMA
45,KPIGREEN,731.05,5.18,65.0,H-SC,5.83,125105.0,-192.0,59125.0,3.76,-0.15,47.26,47.03,141.0,-0.00,0.89,56.64,MH,ATH,POWER


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
70,TANLA,1963.11,-26.90,37.0,H-SC,18.71,196498.0,-74263.0,404216.0,-4.80,-27.43,205.71,121.86,133.0,-0.18,1.41,53.72,AR,ATH,IT
31,HINDZINC,730.22,29.80,52.0,M-LC,10.10,211258.0,6182.0,106390.0,-2.92,3.01,50.36,54.89,52.0,0.06,1.51,27.84,X5K,ATH,METALS
84,WHIRLPOOL,2270.00,-40.03,63.0,M-SC,4.16,101310.0,9812.0,68941.0,-2.43,10.72,68.05,86.07,223.0,0.14,0.72,47.12,XR,NTT,DURABLES
21,COLPAL,3767.14,-3.88,46.0,X-MC,7.99,224230.0,-39135.0,152476.0,-2.31,-14.86,68.00,43.04,84.0,-0.26,1.60,4.10,XY25,ATH,FMCG
39,INFY,2275.00,-21.03,40.0,X-LC,7.49,306954.0,-5934.0,177634.0,-2.07,-1.90,57.87,54.87,3.0,-0.03,2.20,4.63,X40,BTT,IT


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
59,SAIL,228.00,41.14,46.0,M-MC,10.82,222913.0,-2049.0,169704.0,0.60,-0.91,76.13,74.53,192.0,-0.01,1.59,31.05,XY24,BTT,STEEL
45,KPIGREEN,731.05,5.18,65.0,H-SC,5.83,125105.0,-192.0,59125.0,3.76,-0.15,47.26,47.03,141.0,-0.00,0.89,56.64,MH,ATH,POWER
36,INDIAMART,4810.62,-55.57,38.0,H-SC,7.75,123776.0,440.0,131190.0,-0.37,0.36,105.99,106.72,119.0,0.00,0.89,23.82,AR,ATH,MISC


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
77,UJJIVANSFB,60.0,118.20,69.0,H-SC,12.95,137916.0,-4563.0,24080.0,4.84,-3.20,17.46,13.70,163.0,-0.19,0.99,62.78,OX40N,NTT,BANKS
44,KANSAINER,340.0,-66.87,61.0,H-SC,1.93,227925.0,-41742.0,78064.0,-0.00,-15.48,34.25,13.47,138.0,-0.53,1.63,15.89,XY24,NTT,PAINTS
76,TTKPRESTIG,770.0,94.24,44.0,M-SC,1.73,83683.0,-17094.0,17188.0,-0.53,-16.96,20.54,0.09,245.0,-0.99,0.60,9.94,OX40N,NTT,DURABLES
17,CERA,9475.0,-23.39,44.0,H-SC,2.46,139484.0,-36419.0,78446.0,1.04,-20.70,56.24,23.89,149.0,-0.46,1.00,20.46,OX40N,NTT,CERAMICS
65,SIS,528.0,2068.91,52.0,H-SC,3.80,87529.0,-23503.0,47108.0,2.13,-21.17,53.82,21.26,156.0,-0.50,0.63,18.26,OX40N,NTT,MISC


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
54,RECLTD,446.00,45.36,48.0,M-LC,6.05,205838.0,2668.0,39459.0,-0.19,1.31,19.17,20.74,55.0,0.07,1.47,6.93,XY25,NTT,FINANCE
31,HINDZINC,730.22,29.80,52.0,M-LC,10.10,211258.0,6182.0,106390.0,-2.92,3.01,50.36,54.89,52.0,0.06,1.51,27.84,X5K,ATH,METALS
78,UNIONBANK,163.00,-11.63,59.0,M-LC,8.78,160772.0,19932.0,24566.0,3.45,14.15,15.28,31.59,66.0,0.81,1.15,44.39,XY24,NTT,BANKS
11,BANKINDIA,190.00,-22.98,65.0,H-MC,12.57,199248.0,19440.0,92969.0,5.23,10.81,46.66,62.52,88.0,0.21,1.43,48.58,XR,NTT,BANKS
84,WHIRLPOOL,2270.00,-40.03,63.0,M-SC,4.16,101310.0,9812.0,68941.0,-2.43,10.72,68.05,86.07,223.0,0.14,0.72,47.12,XR,NTT,DURABLES


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
11,BANKINDIA,190.00,-22.98,65.0,H-MC,12.57,199248.0,19440.0,92969.0,5.23,10.81,46.66,62.52,88.0,0.21,1.43,48.58,XR,NTT,BANKS
84,WHIRLPOOL,2270.00,-40.03,63.0,M-SC,4.16,101310.0,9812.0,68941.0,-2.43,10.72,68.05,86.07,223.0,0.14,0.72,47.12,XR,NTT,DURABLES
31,HINDZINC,730.22,29.80,52.0,M-LC,10.10,211258.0,6182.0,106390.0,-2.92,3.01,50.36,54.89,52.0,0.06,1.51,27.84,X5K,ATH,METALS
36,INDIAMART,4810.62,-55.57,38.0,H-SC,7.75,123776.0,440.0,131190.0,-0.37,0.36,105.99,106.72,119.0,0.00,0.89,23.82,AR,ATH,MISC
45,KPIGREEN,731.05,5.18,65.0,H-SC,5.83,125105.0,-192.0,59125.0,3.76,-0.15,47.26,47.03,141.0,-0.00,0.89,56.64,MH,ATH,POWER


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
72,TATAMOTORS,1065.0,-53.83,14.0,X-LC,9.58,145037.0,-131486.0,241559.0,0.74,-47.55,166.55,39.81,54.0,-0.54,1.04,2.23,XY24,NTT,AUTO
51,QUESS,986.0,-47.66,37.0,X-SC,38.75,52122.0,-12884.0,163814.0,1.67,-19.82,314.29,232.18,198.0,-0.08,0.37,1.67,XY24,NTT,MISC
39,INFY,2275.0,-21.03,40.0,X-LC,7.49,306954.0,-5934.0,177634.0,-2.07,-1.90,57.87,54.87,3.0,-0.03,2.20,4.63,X40,BTT,IT
9,BAJAJHFL,181.5,-15.11,41.0,X-MC,5.47,178809.0,-21697.0,118854.0,-0.37,-10.82,66.47,48.45,90.0,-0.18,1.28,3.11,X40N,ATH,FINANCE
55,RELAXO,1176.0,-40.17,41.0,X-SC,4.58,81573.0,-63587.0,139514.0,-0.99,-43.80,171.03,52.31,136.0,-0.46,0.58,9.19,X40N,NTT,FOOTWEAR


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
64,SIEMENS,4671.50,-3.68,42.0,H-LC,0.35,154855.0,-31240.0,78713.0,-0.77,-16.79,50.83,25.51,15.0,-0.40,1.11,14.12,AR,ATH,ELECTRICAL
83,VOLTAS,1530.00,3.12,63.0,X-MC,1.57,216360.0,24618.0,13133.0,1.11,12.84,6.07,19.69,99.0,1.87,1.55,20.51,XY25,NTT,AC
28,HAVELLS,2069.16,-6.60,42.0,X-MC,2.38,231488.0,-16243.0,91299.0,-0.51,-6.56,39.44,30.30,92.0,-0.18,1.66,5.86,X40,ATH,ELECTRICAL
41,ITC,452.00,-37.50,60.0,X-LC,2.61,201908.0,1770.0,19121.0,0.18,0.88,9.47,10.44,4.0,0.09,1.44,6.50,X40,NTT,FMCG
33,ICICIGI,2260.25,-13.61,65.0,X-MC,2.65,148577.0,12444.0,18676.0,0.34,9.14,12.57,22.86,91.0,0.67,1.06,22.91,X40,ATH,INSURANCE


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
51,QUESS,986.00,-47.66,37.0,X-SC,38.75,52122.0,-12884.0,163814.0,1.67,-19.82,314.29,232.18,198.0,-0.08,0.37,1.67,XY24,NTT,MISC
72,TATAMOTORS,1065.00,-53.83,14.0,X-LC,9.58,145037.0,-131486.0,241559.0,0.74,-47.55,166.55,39.81,54.0,-0.54,1.04,2.23,XY24,NTT,AUTO
2,ACC,3906.00,-38.50,43.0,X-MC,3.22,183100.0,-54851.0,207507.0,-0.09,-23.05,113.33,64.15,174.0,-0.26,1.31,2.71,XY24,BTT,CEMENT
9,BAJAJHFL,181.50,-15.11,41.0,X-MC,5.47,178809.0,-21697.0,118854.0,-0.37,-10.82,66.47,48.45,90.0,-0.18,1.28,3.11,X40N,ATH,FINANCE
21,COLPAL,3767.14,-3.88,46.0,X-MC,7.99,224230.0,-39135.0,152476.0,-2.31,-14.86,68.00,43.04,84.0,-0.26,1.60,4.10,XY25,ATH,FMCG


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
51,QUESS,986.00,-47.66,37.0,X-SC,38.75,52122.0,-12884.0,163814.0,1.67,-19.82,314.29,232.18,198.0,-0.08,0.37,1.67,XY24,NTT,MISC
55,RELAXO,1176.00,-40.17,41.0,X-SC,4.58,81573.0,-63587.0,139514.0,-0.99,-43.80,171.03,52.31,136.0,-0.46,0.58,9.19,X40N,NTT,FOOTWEAR
12,BATAINDIA,2096.00,-35.15,47.0,X-SC,4.27,96944.0,-31726.0,77022.0,1.54,-24.66,79.45,35.20,219.0,-0.41,0.69,11.27,X40,NTT,FOOTWEAR
16,CAMS,5211.76,-5.83,48.0,X-SC,0.82,107128.0,5124.0,38802.0,0.80,5.02,36.22,43.06,122.0,0.13,0.77,23.00,X40N,ATH,MISC
32,HONAUT,58357.33,-22.75,48.0,X-SC,2.56,108930.0,-19008.0,66142.0,0.00,-14.86,60.72,36.84,143.0,-0.29,0.78,11.52,X40N,ATH,ELECTRICAL


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
73,TCS,4389.97,-29.01,51.0,X-LC,12.73,282996.0,-62960.0,129669.0,1.63,-18.20,45.82,19.28,1.0,-0.49,2.02,4.62,X40,ATH,IT
39,INFY,2275.00,-21.03,40.0,X-LC,7.49,306954.0,-5934.0,177634.0,-2.07,-1.90,57.87,54.87,3.0,-0.03,2.20,4.63,X40,BTT,IT
41,ITC,452.00,-37.50,60.0,X-LC,2.61,201908.0,1770.0,19121.0,0.18,0.88,9.47,10.44,4.0,0.09,1.44,6.50,X40,NTT,FMCG
82,VBL,671.64,-19.29,49.0,X-LC,10.53,290832.0,-25010.0,137680.0,-1.24,-7.92,47.34,35.67,5.0,-0.18,2.08,4.84,X40N,ATH,FMCG
1,ABB,7934.00,-39.87,53.0,H-LC,6.89,251472.0,-10147.0,129357.0,0.71,-3.88,51.44,45.57,7.0,-0.08,1.80,7.00,AR,NTT,ELECTRICAL


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
6,ASIANTILES,137.00,7044.44,67.0,L-SC,6.47,78397.0,-15413.0,92030.0,3.67,-16.43,117.39,81.67,269.0,-0.17,0.56,51.82,XR,NTT,CERAMICS
20,COFFEEDAY,80.00,-45.67,50.0,L-SC,29.64,81696.0,-31853.0,71901.0,0.42,-28.05,88.01,35.27,268.0,-0.44,0.58,99.95,XR,NTT,HOTELS
48,MASFIN,398.61,-20.05,47.0,H-SC,9.44,90975.0,-7005.0,28612.0,1.97,-7.15,31.45,22.05,152.0,-0.24,0.65,32.24,XR,ATH,FINANCE
84,WHIRLPOOL,2270.00,-40.03,63.0,M-SC,4.16,101310.0,9812.0,68941.0,-2.43,10.72,68.05,86.07,223.0,0.14,0.72,47.12,XR,NTT,DURABLES
45,KPIGREEN,731.05,5.18,65.0,H-SC,5.83,125105.0,-192.0,59125.0,3.76,-0.15,47.26,47.03,141.0,-0.00,0.89,56.64,MH,ATH,POWER


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
60,SAMMAANCAP,326.0,-189.77,73.0,M-SC,19.77,154260.0,5040.0,139143.0,2.78,3.38,90.20,96.62,208.0,0.04,1.10,66.94,XY25,NTT,FINANCE
77,UJJIVANSFB,60.0,118.20,69.0,H-SC,12.95,137916.0,-4563.0,24080.0,4.84,-3.20,17.46,13.70,163.0,-0.19,0.99,62.78,OX40N,NTT,BANKS
68,SURYODAY,240.0,66.67,61.0,H-SC,11.16,151980.0,-27091.0,92814.0,5.46,-15.13,61.07,36.71,135.0,-0.29,1.09,50.79,XR,NTT,BANKS
4,ANGELONE,3033.0,15.82,70.0,X-SC,8.29,205254.0,14248.0,43452.0,0.26,7.46,21.17,30.21,157.0,0.33,1.47,29.90,X40N,NTT,FINANCE
67,STARHEALTH,761.0,20.77,65.0,H-SC,9.77,269008.0,-12115.0,144968.0,-1.88,-4.31,53.89,47.26,144.0,-0.08,1.92,44.13,XY24,NTT,INSURANCE


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,19.51
1,25,44.12
2,50,75.80


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    44.46
LC    33.79
MC    21.72
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     30.56
X40      14.77
XY25     12.38
XR       11.06
X40N     10.24
AR        7.99
OX40N     7.67
X200      1.80
X5K       1.51
SR        1.10
MH        0.89
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    25.54
X-LC    22.85
X-MC    15.98
M-SC    12.78
M-LC     5.20
H-LC     4.71
X-SC     4.66
H-MC     3.84
M-MC     1.59
L-SC     1.48
L-LC     1.03
L-MC     0.31
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,14.12,-3.87,38.10
IT,12.62,-19.20,83.54
FINANCE,12.19,-10.86,61.12
MISC,6.91,-19.07,85.05
BANKS,6.57,-8.82,66.06
PAINTS,5.67,-16.25,33.58
ELECTRICAL,5.35,-10.26,48.95
INSURANCE,3.94,-0.14,37.58
AUTO,2.84,-45.32,114.78


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3299428.0,22
XR,1364030.0,14
AR,1214727.0,9
X40,769695.0,10
X40N,595606.0,8
OX40N,574758.0,10
XY25,449229.0,8
SR,266658.0,2
X5K,106390.0,1


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3430434.0,25
M-SC,1254423.0,15
X-MC,1091658.0,11
X-LC,1039897.0,13
X-SC,528746.0,6
H-MC,343299.0,3
H-LC,281955.0,3
M-LC,281410.0,4
L-SC,253007.0,3


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1176349.0      6
           AR         843887.0      5
           XR         818337.0      7
M-SC       XY24       685879.0      6
X-MC       XY24       591967.0      4
X-LC       X40        477445.0      5
           XY24       308378.0      2
X-SC       X40N       287910.0      4
H-SC       SR         266658.0      2
           OX40N      266078.0      4
X-MC       X40        215228.0      4
H-LC       AR         208070.0      2
X-LC       X40N       188842.0      3
H-MC       XY24       178771.0      1
M-MC       XY24       169704.0      1
X-MC       XY25       165609.0      2
L-SC       XR         163931.0      2
X-SC       XY24       163814.0      1
M-SC       AR         162770.0      2
           OX40N      148045.0      4
           XY25       139143.0      1
X-MC       X40N       118854.0      1
M-SC       XR         118586.0      2
M-LC       XR         110995.0      1
           X5K        106390.0      1
H-MC       XR          92969.0      1
L-SC       OX40N       89076.0      1
X-SC       X40         77022.0      1
H-LC       X200        73885.0      1
H-MC       OX40N       71559.0      1
X-LC       XY25        65232.0      3
L-MC       XR          59212.0      1
H-SC       MH          59125.0      1
L-LC       XY25        39786.0      1
M-LC       XY25        39459.0      1
           XY24        24566.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 28.0 seconds
